## Classification

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

## Data preparation

In [12]:
df = pd.read_csv('course_lead_scoring.csv')
df.head()

,429: Too Many Requests
For more on scraping GitHub and how it may affect your rights,please review our Terms of Service (https://d...
